In [ ]:
from langchain.tools import TavilySearchResults
# llm_config.py
import os
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()


embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-002",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")




🧰 ## Step 1: Define the Tools
1. Web Search Agent
Utilize LangChain's integration with web search APIs like Tavily or SerpAPI.

In [2]:
tavily_search = TavilySearchResults(max_results=1)

## 2. PDF Search Tool Agent
Implement a Retrieval-Augmented Generation (RAG) system to search through PDF documents.

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.tools import Tool
from langchain_core.tools import tool

# Load and split the PDF
loader = PyPDFLoader("sample_pdf_files/deep-learning-r-tutorial.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

vectorstore = Chroma.from_documents(texts, embeddings)


@tool
def web_search_tool(query: str) -> str:
    """Use the Tavily Search API to retrieve current information."""
    return tavily_search.invoke({"query": query})


@tool
def pdf_search_tool(query: str) -> str:
    """Search content inside uploaded PDFs using semantic similarity."""
    docs = vectorstore.similarity_search(query, k=1)
    return "\n".join([doc.page_content for doc in docs])


## 3. Database Agent
Create a custom function to query your database based on user_id.

In [5]:
from langchain.tools import Tool
import sqlite3

@tool
def get_account_info(user_id: int) -> dict:
    """Fetch account info from the database for a given user_id."""
    conn = sqlite3.connect('sample_users.db')
    cursor = conn.cursor()

    cursor.execute('''
        SELECT user_id, name, account_balance, membership_status
        FROM user_account_details
        WHERE user_id = ?
    ''', (user_id,))
    
    row = cursor.fetchone()
    conn.close()

    if row:
        return {
            "user_id": row[0],
            "name": row[1],
            "account_balance": row[2],
            "membership_status": row[3]
        }
    else:
        return {"error": "User not found"}
    
# Wrap the function with LangChain's Tool
get_account_info_tool = Tool.from_function(
    func=get_account_info,
    name="get_account_info",
    description="Fetches account information from the database given a user_id."
)

In [6]:
from langchain.agents import initialize_agent, AgentType

tools = [web_search_tool,pdf_search_tool,get_account_info]

# initialize agent with tools
# Initialize the agent with the tool
agent_executer = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True,
    handle_parsing_errors=True
)

C:\Users\Sayantan\AppData\Local\Temp\ipykernel_2852\554539521.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executer = initialize_agent(


In [14]:
response = agent_executer("Whats my account balance, my user id is 72")
response



> Entering new AgentExecutor chain...
Thought: I need to retrieve the account information for user ID 72 to find the account balance.  I'll use the `get_account_info` tool.
Action: get_account_info
Action Input: 72
Observation: {'user_id': 72, 'name': 'Bugs Bunty', 'account_balance': '£77.50', 'membership_status': 'Gold Executive'}
Thought:Thought: I now know the final answer.
Final Answer: Your account balance is £77.50.

> Finished chain.


{'input': 'Whats my account balance, my user id is 72',
 'output': 'Your account balance is £77.50.'}

In [15]:
response = agent_executer("Whats the current weather in kalna?")
response



> Entering new AgentExecutor chain...
Thought: I need to find the current weather in Kalna.  A web search is the best way to get this information.
Action: web_search_tool
Action Input: "current weather in Kalna"
Observation: [{'title': 'Weather in Kalna', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Kalna', 'region': 'West Bengal', 'country': 'India', 'lat': 23.2167, 'lon': 88.3667, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1744384463, 'localtime': '2025-04-11 20:44'}, 'current': {'last_updated_epoch': 1744383600, 'last_updated': '2025-04-11 20:30', 'temp_c': 27.3, 'temp_f': 81.2, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 15.0, 'wind_kph': 24.1, 'wind_degree': 204, 'wind_dir': 'SSW', 'pressure_mb': 1005.0, 'pressure_in': 29.67, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 69, 'cloud': 11, 'feelslike_c': 29.6, 'feelslike_f': 85.4, 'windchill_c': 27.3, 'windchill_f'

{'input': 'Whats the current weather in kalna?',
 'output': 'The current weather in Kalna is clear with a temperature of 27.3 degrees Celsius.'}

In [16]:
response = agent_executer("Whats is Deep Learning in the pdf")



> Entering new AgentExecutor chain...
Thought: I need to search the uploaded PDFs for information about Deep Learning.
Action: pdf_search_tool
Action Input: "Deep Learning"
Observation: 1. Introduction to Deep Learning
Deep Learning is a subset of machine learning that uses artificial neural networks with multiple layers to learn representations of data. These deepneural networks have revolutionized many fields, including computer vision, natural language processing, and speech recognition.
Key concepts in deep learning include:
Neurons and layersActivation functionsBackpropagationGradient descentLoss functions
Thought:Question: Whats is Deep Learning in the pdf
Thought: I need to search the uploaded PDFs for information about Deep Learning.
Action: pdf_search_tool
Action Input: "Deep Learning"
Observation: 1. Introduction to Deep Learning
Deep Learning is a subset of machine learning that uses artificial neural networks with multiple layers to learn representations of data. These de

In [17]:
response

{'input': 'Whats is Deep Learning in the pdf',
 'output': 'Deep Learning is a subset of machine learning that uses artificial neural networks with multiple layers to learn representations of data.  These deep neural networks have revolutionized many fields, including computer vision, natural language processing, and speech recognition.  Key concepts include neurons and layers, activation functions, backpropagation, gradient descent, and loss functions.'}